In [17]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx

In [18]:
filename = '../data/nextbike/rents_returns_by_bike_Dresden_01-03_09-10.24.pkl'
#filename = '../data/nextbike/rents_returns_by_bike_Freiburg_06-07.23_09-10.24.pkl'
df_DD = pd.read_pickle(filename)

In [19]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option('display.max_colwidth', 400)

In [ ]:
df_DD.head()

In [ ]:
df_DD.date.sort_values().unique()

In [ ]:
df_DD.date.iloc[0]

In [ ]:
df_DD.info()

In [ ]:
df_DD.sort_values("datetime")

In [25]:
#flt_start = df_DD.date == pd.to_datetime("2024-01-01").date()
flt_start = df_DD.date == pd.to_datetime("2023-06-01").date()

#df_DD.loc[flt_start]
start_stations = df_DD.loc[flt_start, "station_number"].dropna().unique().tolist()

flt_end = df_DD.date == pd.to_datetime("2024-10-31").date()
#df_DD.loc[flt_start]
end_stations = df_DD.loc[flt_end, "station_number"].dropna().unique().tolist()



In [ ]:
start_stations = set(start_stations)
end_stations = set(end_stations)
len(start_stations), len(end_stations)

In [ ]:
len(start_stations-end_stations), len(end_stations-start_stations)


In [ ]:
df_DD.columns

In [29]:
start_stations_coor = df_DD.loc[flt_start, ["station_number", "lat", "lng"]].dropna(subset="station_number")[["lat", "lng"]].drop_duplicates()
end_stations_coor = df_DD.loc[flt_end, ["station_number", "lat", "lng"]].dropna(subset="station_number")[["lat", "lng"]].drop_duplicates()

In [30]:
merged_stations= start_stations_coor.merge(end_stations_coor, how='outer', indicator = True )

In [ ]:
merged_stations.head()

In [ ]:
df_DD[df_DD.rent_or_return=='returns'].groupby(['time_stamp', 'uid', 'name']).size().sort_values(ascending=False).head(40)

In [343]:
# df_DD[df_DD.rent_or_return=='returns'].groupby(['time_stamp', 'name']).size().sort_values(ascending=False).head(40)

In [344]:
#df_one_bike = df_DD[df_DD.bike_number==32975].sort_values("datetime")

In [345]:
# # Sample list of coordinates (latitude, longitude) in a city (e.g., New York)
# coordinates = [
#     (40.7128, -74.0060),  # Manhattan
#     (40.730610, -73.935242),  # Queens
#     (40.650002, -73.949997),  # Brooklyn
#     (40.8448, -73.8648),  # The Bronx
#     (40.5795, -74.1502)  # Staten Island
# ]

# # Convert to GeoPandas DataFrame
# geometry = [Point(lon, lat) for lat, lon in coordinates]
# gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")  # Use WGS84 latitude/longitude

# # Reproject to a metric CRS for plotting with basemap
# gdf = gdf.to_crs(epsg=3857)

# # Plot the points on a map
# ax = gdf.plot(marker='o', color='red', markersize=50, figsize=(10, 10))

# # Add a basemap (e.g., OpenStreetMap)
# ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# # Add labels for each point
# for x, y, coord in zip(gdf.geometry.x, gdf.geometry.y, coordinates):
#     plt.text(x, y, f'{coord}', fontsize=8, ha='left')

# # Customize and display the map
# plt.title("Coordinates in a City", fontsize=16)
# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# plt.show()


In [ ]:
start_stations_coor.columns

In [ ]:
merged_stations.head()

In [ ]:
# Sample list of coordinates (latitude, longitude) in a city (e.g., New York)

# Convert to GeoPandas DataFrame
geometry = [Point(xy) for xy in zip(merged_stations.lng, merged_stations.lat)]
gdf = gpd.GeoDataFrame(merged_stations, geometry=geometry, crs="EPSG:4326")  

# Reproject to a metric CRS for plotting with basemap
gdf = gdf.to_crs(epsg=3857)

gdf_l = gdf[gdf['_merge'] == 'left_only']
gdf_r = gdf[gdf['_merge'] == 'right_only']
gdf_b = gdf[gdf['_merge'] == 'both']

# Plot the points on a map
fig, ax = plt.subplots(figsize=(10, 10))

gdf_l.plot(ax=ax, marker=0, color='red', markersize=50, label='Station was removed over the timeline')
gdf_r.plot(ax=ax, marker='+', color='green', markersize=50, label='Station appeared over the timeline')
gdf_b.plot(ax=ax, marker='o', color='black', markersize=50, label='Station existed throughout the timeline')

# Add a basemap (e.g., OpenStreetMap)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

# Add labels for each point
# for x, y, coord in zip(gdf.geometry.x, gdf.geometry.y, coordinates):
#     plt.text(x, y, f'{coord}', fontsize=8, ha='left')
ax.axis('off')
# Customize and display the map
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.savefig('../data/plots/FB_station_dynamics.png', format='png')
plt.show()
